## Transformers From Scratch

Notebook where I build a Transformer from scratch using PyTorch neural networks.

### Read In Input File

In [2]:
#read in shakespear input file
with open('../notebooks/data/input.txt', 'r', encoding = 'utf-8') as file:
    text = file.read()

In [3]:
#get all the unique characters that occur in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65
